In [1]:
from wikipedia_tools import tools_mapping
from wiki_agent import WikipediaAgent

In [ ]:
file_save_format: str = """
Question: {user_question}
Topic: {topic}
Summary: {summary}
"""  # format we expect agent to follow when saving results

system_prompt = f"""You are an expert researcher agent that leverages Wikipedia to answer questions. 
You are not to answer questions without using a tool or using information resulting from a tool. 

Tools:
You have access to these tools:
1. search_wikipedia(query: str)  
  - Always provide a non-empty query string.

2. save_results_to_path(content: str, file_name: str, file_directory: str)  
  - Always provide non-empty content and a valid file_name.
    - Content must be the summary you generated.
  - If no file_name is given, use "summary.txt".
  - If no file_directory is given, use "." (the current working directory).
  - Never output empty strings for parameters.
  - The **content must always follow this format**:  
    {file_save_format}  
    - {{user_question}} is the original user query.
    - {{topic}} is the main subject extracted (e.g., "Sudanese Civil War").
    - {{summary}} is the most recent findings or research results (never empty).  

3. get_top_k_keywords(content: str, k: int)  
  - Always provide non-empty content and a valid integer k.

4. read_results_from_path(path: str)  
  - Always provide a valid non-empty file path.

Important:
- Do NOT output empty strings ("") for any parameter.
- You must provide the keyword arguments in the parameters json, i.e. parameters={{"query":"I am asking a question."}}

Output guidelines:
You must always respond in the following JSON format:
{{"intent": ..., "status": ..., "function": {{"name": ..., "parameters": ...}}}}

- "status" is "in_progress" if another tool call is required, otherwise "done".
- Do not omit any field.

Do not include any other text outside of the JSON.
Always respond with valid JSON using standard double quotes ".
"""


In [3]:
agent = WikipediaAgent(
  model_name="llama3.1",
  tools=tools_mapping,
  system_prompt=system_prompt,
)

In [4]:
summary = agent.run(query="What is the status of the Sudanese Civil War in August 2025?")

Step 1: intent='information' status='in_progress' function={'name': 'search_wikipedia', 'parameters': {'query': 'sudanese civil war current situation august 2025'}} tool_result=None
Step 7: intent='search_wikipedia' status='done' function={'name': 'search_wikipedia', 'parameters': {'query': 'sudanese civil war current situation august 2025'}} tool_result=None


In [5]:
summary

'The current situation in the Sudanese civil war as of August 2025 is that the conflict continues with no lasting ceasefire reached despite renewed negotiations. The Rapid Support Forces (RSF) control most of Darfur, while the Sudanese Armed Forces (SAF) regained momentum in early 2024 and eventually retaken Khartoum by March 2025.'

In [6]:
for step in agent.most_recent_trace:
    print(step)

intent='information' status='in_progress' function={'name': 'search_wikipedia', 'parameters': {'query': 'sudanese civil war current situation august 2025'}} tool_result={'success': True, 'result': "Page: Sudanese civil war (2023–present)\nSummary: A civil war began on 15 April 2023 between two rival factions of the military government of Sudan. The conflict involves the Sudanese Armed Forces (SAF), led by General Abdel Fattah al-Burhan, and the paramilitary Rapid Support Forces (RSF), commanded by Mohamed Hamdan Dagalo (commonly known as Hemedti), who also leads the broader Janjaweed coalition.  Several smaller armed groups have also taken part. Fighting has been concentrated in the capital, Khartoum, where the conflict began with large-scale battles, and in the Darfur region. Many civilians in Darfur have been reported dead as part of the Masalit massacres, which have been described as ethnic cleansing or genocide. Sudan has been described as facing the world's worst humanitarian cris

In [7]:
save_summary = agent.run(query="Can you save your findings to a text file in the results directory?")

Step 1: intent='save_results' status='done' function={'name': 'save_results_to_path', 'parameters': {'content': '', 'file_name': '', 'file_directory': ''}} tool_result=None


In [8]:
save_summary

'You need to specify a file name when saving the results. The conflict in Sudan continues with no lasting ceasefire reached despite negotiations.'

In [9]:
for step in agent.most_recent_trace:
    print(step)

intent='save_results' status='done' function={'name': 'save_results_to_path', 'parameters': {'content': 'The current situation in the Sudanese civil war as of August 2025 is that the conflict continues with no lasting ceasefire reached despite renewed negotiations. The Rapid Support Forces (RSF) control most of Darfur, while the Sudanese Armed Forces (SAF) regained momentum in early 2024 and eventually retaken Khartoum by March 2025.', 'file_name': '', 'file_directory': ''}} tool_result={'success': False, 'result': "[Errno 2] No such file or directory: ''"}


In [10]:
sudan_summary = agent.run(query="Save your sudan summary to ./results/sudan_summary.txt")

Step 4: intent='save_sudan_summary' status='done' function={'name': 'save_results_to_path', 'parameters': {'content': "question: sudanese civil war\ntopic: sudanese civil war\nsummary: the second sudanese civil war (1983–2005) was a conflict in the sudan that began as a mutiny of the sudan people's liberation army (spla) among the non-arab, african population of south sudan.\n", 'file_name': 'wiki_findings_1.txt', 'file_directory': './results'}} tool_result=None
File saved successfully at ./results/wiki_findings_1.txt.


In [11]:
content = agent.run(query="Read the contents of ./random_txt.txt and summarize it.")

Step 1: intent='read_file' status='in_progress' function={'name': 'read_results_from_path', 'parameters': {'path': './random_txt.txt'}} tool_result=None
File read successfully.
Step 3: intent='read results from file' status='done' function={'name': 'read_results_from_path', 'parameters': {'path': './random_txt.txt'}} tool_result=None
File read successfully.


In [12]:
content

"The content of the file './random_txt.txt' is a passage about Lorem Ipsum's origins and history."

In [13]:
keywords = agent.run(query="Extract the top 5 keywords from the contents of ./random_txt.txt.")

Step 1: intent='keyword extraction' status='done' function={'name': 'get_top_k_keywords', 'parameters': {'content': 'read_results_from_path', 'k': '5'}} tool_result=None


In [14]:
keywords

"The tool call failed due to an invalid input type for the 'k' parameter. It requires a numeric value."